In [2]:
!pip install -U ragas datasets langchain openai tiktoken LangchainLLM openai

ERROR: Could not find a version that satisfies the requirement LangchainLLM (from versions: none)
ERROR: No matching distribution found for LangchainLLM


In [9]:
# Chat GPT-4o모델로 RAGAS평가를 위한 API 키 설정
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI")

# Colab환경에서 파일 업로드
from google.colab import files
import pandas as pd

uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0], encoding="cp949")

# LLM 답변만 추출
answers = df["llm_answer"].dropna().tolist()

# context / reference 정의
question = "조건의 건국과정"

answer = """
    당신은 모든 문서에서만 정보를 추출하여 답변해야 합니다.
    당신은 모든 문서를 기반으로 역사적 사실만 전달해야하는 역사선생님 입니다.
    검색된 모든 문서 내용에 없는 정보는 절대 추측하지 말고 "해당 정보는 제공되지 않았습니다"라고 답하십시오.
    주어진 문서에 없는 추측은 **절대 금지**입니다.
    답변은 한번만 출력하도록 하십시오.
    답변은 완전한 문장으로 답하시오.
    반복되는 문장없이 출력하시오.
    한 줄에 한 문장이상 넘지않도록 출력하고 이어서 아랫줄로 넘어가세요.
    문서에 복수 해석이 가능한 경우에는 모두 서술하시오.
    예: "아마 ~일 것이다" 형태의 표현은 금지.
    최대 10문장이내로 답변하세요.
    답변의 각 문장은 어떤 문서의 어느 부분에서 왔는지 정확히 출처를 함께 달아줘. 만약 출처가 없으면 그 내용은 빼고, 추론 없이 대답해줘.
    질문에 대해서 모든 문서를 검색해서 정확히 알려줘. 한 문장마다 출처가 있어야 하고, 없는 문장은 쓰지 마.

    모든 문서를 참고하여 '기승전결' 구조로 수업자료를 작성하시오.

    조건:
    1. "기(起) - 발단", "승(承) - 전개", "전(轉) - 전환", "결(結) - 마무리" 형식으로 구분하여 작성
    2. 각 항목은 간결하게 2~3문장
    3. 마지막에 '전체 맥락 요약' 항목을 추가
    4. 교과서 스타일의 설명체로 작성
"""


contexts = ["""
조선의 건국
⑴ 위화도 회군(1388): 명의 철령 이북 땅 요구 → 요동 정벌 추진 → 이성계가 위화도에서 회군 → 이성계의 국정 장악
⑵ 과전법 실시(1391)
① 내용: 전현직 관료에게 수조권을 나누어 준 제도
② 결과: 신진 사대부의 경제적 기반 마련, 국가 재정 확충
⑶ 신진 사대부의 분화: 새 왕조 개창을 주장하는 세력과 고려 왕조 유지를 주장하는 세력 간 의견 차이 발생
⑷ 조선 건국(1392): 정도전 등이 이성계를 왕으로 추대
⑸ ‌한양 천도(1394): 경복궁을 중심으로 종묘와 사직단 및 인 · 의 · 예 · 지 등 유교 덕목을 넣은 건축물 세움.

•위화도 회군 이후 조선이 건국되는 과정을 연표를 통해 정리한다.
•고려 말이라는 시대 상황을 고려하여 조선의 건국을 평가할 수 있도록 한다.

역성혁명의 형식을 빌려 이성계를 태조로 하는 조선왕조가 개창되었다(1392).
결국, 공민왕의 개혁정치로부터 시작된 정세의 변화가 신흥사대부와 신흥무장의 정치적 성장과 새로운 왕조의 성립으로 귀결된 셈이다.
그러나 조선의 건국은 일찍이 무신란으로 표출된 고려사회의 모순과 부마국체제 아래 축적된 혼란과 폐단을 해결하고 수습하려는 몸부림의 결과로서의 의미를 지닌다고 하겠다.
새로이 성립된 조선은 사상적으로 억불양유(抑佛揚儒)를 표방하고, 대외관계면에서 명에 대한 사대정책(事大政策)을 취하였다.

1. 조선의 건국
⑴ 위화도 회군(1388): 명의 철령 이북 땅 요구 → 요동 정벌 추진 → 이성계가 위화도에서 회군 → 이성계의 국정 장악
⑵ 과전법 실시(1391): 전현직 관료에게 수조권을 나누어 준 제도 → 신진 사대부의 경제적 기반 마련, 국가 재정 확충
⑶ 조선 건국(1392): 정도전 등이 이성계를 왕으로 추대”민족대백과_2
"""]

reference = """
기(起) - 발단
명의 철령위 설치 통보로 군사적 긴장이 고조되자, 고려는 요동 정벌을 결정하였다(출처:민족대백과_4)
그러나 이성계는 위화도에서 회군하여 정권을 장악하고 개혁 정치를 추진하였다(출처:민족대백과_4)

승(承) - 전개
개혁의 일환으로 1391년 과전법이 시행되어 전·현직 관리에게 수조권이 지급되었다(출처:민족대백과_4)
이 조치는 권문세족의 경제력을 약화시키고 신진 사대부의 정치 참여를 가능하게 하였다(출처:민족대백과_4)

전(轉) - 전환
신진 사대부는 고려 왕조 유지파와 새 왕조 창건파로 분화되었다(출처:민족대백과_4)
정도전과 조준 등은 이성계를 왕으로 추대하며 1392년 조선을 건국하였다(출처:민족대백과_4)

결(結) - 마무리
정도전은 유교를 통치 이념으로 삼고 궁궐·종묘·사직 등을 건설하며 새 왕조의 제도를 설계하였다(출처:민족대백과_4)
조선은 중앙 집권적 관료제를 갖춘 유교 국가로 출범하였다(출처:민족대백과_4)

전체 맥락 요약
조선의 건국은 위화도 회군을 통해 정권을 장악한 이성계 세력이 과전법을 통해 신진 사대부를 결집시키고, 정도전의 주도로 새로운 유교적 질서를 마련하면서 이루어졌다(출처:민족대백과_4)
"""

# RAGAs 평가용 모델 및 평가 지표 import
from langchain.chat_models import ChatOpenAI
from ragas import evaluate
from ragas.metrics import (
    faithfulness,         # 사실성
    answer_relevancy,     # 정답 관련성
    context_recall,       # 문맥 재현율
    context_precision     # 문맥 정밀도
)

llm = llm=ChatOpenAI(model="chatgpt-4o-latest", temperature=0)
evaluate.llm = llm

# RAGAs 평가용 Dataset 구성
from datasets import Dataset
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

dataset = Dataset.from_dict({
    "question": [question] * len(answers),
    "answer": answers,
    "contexts": [contexts] * len(answers),
    "reference": [reference] * len(answers),
})

# 평가 실행
result = evaluate(
    dataset=dataset,
    metrics=[faithfulness, answer_relevancy, context_recall, context_precision],
)

# 결과 반영
ragas_df = result.to_pandas()

# 각 평가 지표별 점수 출력
print("각 평가 지표별 점수 확인:")
print(ragas_df[["faithfulness", "answer_relevancy", "context_recall", "context_precision"]])

# 업로드 파일에 각 평가 지표 결과 추가
df.loc[df["llm_answer"].notnull(), "faithfulness"] = ragas_df["faithfulness"]
df.loc[df["llm_answer"].notnull(), "answer_relevancy"] = ragas_df["answer_relevancy"]
df.loc[df["llm_answer"].notnull(), "context_recall"] = ragas_df["context_recall"]
df.loc[df["llm_answer"].notnull(), "context_precision"] = ragas_df["context_precision"]

# 평균결과 추가
ragas_df["ragas"] = ragas_df[["faithfulness", "answer_relevancy", "context_recall", "context_precision"]].mean(axis=1).round(4)
df.loc[df["llm_answer"].notnull(), "ragas"] = ragas_df["ragas"]

# 평가결과 저장
df.to_csv("ragas_result_with_score.csv", index=False, encoding="utf-8-sig")
print("저장확인")

Saving llm_as_a_judge.csv to llm_as_a_judge (5).csv


Evaluating:   0%|          | 0/144 [00:00<?, ?it/s]

각 평가 지표별 점수 확인:
    faithfulness  answer_relevancy  context_recall  context_precision
0       1.000000          0.794325             1.0                1.0
1       0.714286          0.795563             1.0                1.0
2       0.600000          0.798040             1.0                1.0
3       0.916667          0.795563             1.0                1.0
4       0.785714          0.794325             1.0                1.0
5       0.166667          0.796536             1.0                1.0
6       1.000000          0.800948             1.0                1.0
7       0.700000          0.000000             1.0                1.0
8       1.000000          0.807293             1.0                1.0
9       1.000000          0.807293             1.0                1.0
10      0.647059          0.796615             1.0                1.0
11      0.882353          0.794511             1.0                1.0
12      0.666667          0.808716             1.0                1.0
13  